This notebook is a first attempt to translate the FORTRAN90 code from MTV2016 into python
and to replicate the main results of this paper.

In [1]:
import numpy as np
from scipy.stats import lognorm

import matplotlib.pyplot as plt
plt.style.use(['dark_background'])

In [2]:
#Paramters for the piece-rate version

delta = 0.000226
lamb_e = 0.7048178
k = 22.9931
sigma_z = 0.0
rhorho1 = 4.296953
rho2 = 0.065518
zlearningprob = 0.26398515
b = 2.3105359
theta9 = 0.0
theta10 = 0.0
zshockprob = 0.0093854   


#    IF (npar>=10) THEN
#    rho3=theta(9)  
#    rho4=theta(10)  
#    ELSE IF (npar==8) THEN
#    rho3=0.0_8
#    rho4=0.0_8
#    ELSE 

In [92]:
# sigma_z = 0.2
# mu_temp = np.exp(-0.5 * sigma_z**2) # this guarantess that the mean of the distribution is 1

# x = np.linspace(0, 4, 100)
# plt.plot(x, lognorm.cdf(x, sigma_z, loc=0, scale=mu_temp))
# plt.plot(x, lognorm.pdf(x, sigma_z, loc=0, scale=mu_temp))
# plt.show()

In [88]:
# Match shock z.
# First, populating the match productivity shock vector, if zpts_first is >1

zpts_first = 51 # basic grid. 102 in MTV2016
zpts = zpts_first

# zvector are the actual values of the match quality, dim(zpts)
# ztransmatrix is the transition matrix, dim(zpts, zpts)

# Obtain a discrete LOGNORMAL probability distr for z

zprobcdf = np.zeros((zpts))
zprobpdf = np.zeros((zpts))
zvector = np.zeros((zpts))

sigma_z = 0.2 # CHANGE OF PARAMTER: sigma_z must be positive
mu_temp = np.exp(-0.5 * sigma_z**2) # mean of the lognormal distr, s.t. mean of the distr is 1

shape = sigma_z

resolution = 1000 # Resolution should be much higher than no. of bins (resolution=10000)
xstep = 1 / resolution

xvalue = xstep # move along the interval of z's, starting close to 1
tempreal = 0.5 * xstep * lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) # this value is used to weight the z-values
# properly within the bin
bincounter = 1 # start with the first bin
binsize = 1 / (zpts - 1) # binsize

# zprobcdf contains the probabilities of the idiosyncratic shock, dimension zpts

# First fill all but the last coordinate
while bincounter < zpts - 1:
    if lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) >= ((bincounter + 0) * binsize): # if the bin is full..
        zprobcdf[bincounter] = lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) # add to the cdf
        if bincounter == 1:
            zprobpdf[bincounter] = zprobcdf[bincounter]
            zvector[bincounter] = tempreal / zprobpdf[bincounter]
        else:
            zprobpdf[bincounter] = zprobcdf[bincounter] - zprobcdf[bincounter - 1]
            zvector[bincounter] = tempreal / zprobpdf[bincounter]
        
        tempreal = 0 # reset the average counter 
        bincounter += 1
        
#         while lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) >= (bincounter * binsize):
#             zvector[bincounter] = xstep
#             zprobcdf[bincounter] = lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp)
#             zprobpdf[bincounter] = 0
#             bincounter += 1
            
    else:
        while lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) < ((bincounter + 0) * binsize):
            xvalue = xvalue + xstep
            tempreal = tempreal + (xvalue - 0.5 * xstep) * (lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) \
                                                        - lognorm.cdf(xvalue - xstep, shape, loc=0, scale=mu_temp))
# # last gridpoint

zprobcdf[zpts-1] = 1
zprobpdf[zpts-1] = 1 - zprobcdf[zpts - 2]

tempreal = 0
xstep = xvalue / (zpts * 5)

counter1 = 1
while counter1 < 11:
    xvalue = xvalue + xstep
    tempreal = tempreal + (xvalue - 0.5 * xstep) * (lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) \
                                                        - lognorm.cdf(xvalue - xstep, shape, loc=0, scale=mu_temp))
    counter1 += 1
    
zvector[zpts-1] = tempreal / (lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) - zprobcdf[zpts - 2])

In [152]:
# Z-PRODUCTIVITY TRANSITION MATRICES

# INITIAL LEARNING, same for all options

ztransmatrix = np.zeros((zpts, zpts))
zlearningprob = 0.26398515 # from MTV2016

ztransmatrix[0, 0] = 0
for ind in range(1, zpts):
    ztransmatrix[0, ind] = zlearningprob * zprobpdf[ind]
    
tempreal = np.sum(ztransmatrix[0, :])
ztransmatrix[0, 0] = 1 - tempreal

# Permanent z's once learned
for ind in range(1, zpts):
    ztransmatrix[ind, ind] = 1

In [ ]:
# Worker productivity - ALL WORKER IDENTICAL WHEN BORN

ypts_initial = 1    # twenty productivity types
ypts = 1  # making it even 

ymin = 0.1
ymax = 1.5          # keep symmetry around 1

y_aux_pts = 100           # to construct the normal distribution
y_aux_vector = np.array((y_aux_pts))

y[ypts] = 1
yprobcdf[ypts] = 1
yprobpdf[ypts] = 1